<a href="https://colab.research.google.com/github/Purugru/Chad-Bod/blob/main/gpt2_BOMB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install torch
!pip install -U PyPDF2
!pip install python-docx
!pip install accelerate -U
!pip install tensorflow

In [2]:
import os
import re
from PyPDF2 import PdfReader
import docx
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

Code to read PDF

In [3]:
def read_pdf(file_path):
  with open(file_path, "rb") as file:
    pdf_reader = PdfReader(file)
    text = ""
    for page_nun in range(len(pdf_reader.pages)):
      text += pdf_reader.pages[page_nun].extract_text()
  return text

def read_documents_from_directory(directory):
    combined_text = ""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if filename.endswith(".pdf"):
            combined_text += read_pdf(file_path)
#        elif filename.endswith(".docx"):
#            combined_text += read_word(file_path)
#        elif filename.endswith(".txt"):
#            combined_text += read_txt(file_path)
    return combined_text


In [4]:
!find /content/drive/MyDrive/CHATBOT/ -name "bottenheimer.pdf"

/content/drive/MyDrive/CHATBOT/bottenheimer.pdf


In [5]:
def train_chatbot(directory, model_output_path, train_fraction=0.8):
    # Read documents from the directory
    combined_text = read_documents_from_directory(directory)
    combined_text = re.sub(r'\n+', '\n', combined_text).strip()  # Remove excess newline characters

    # Split the text into training and validation sets
    split_index = int(train_fraction * len(combined_text))
    train_text = combined_text[:split_index]
    val_text = combined_text[split_index:]

    # Save the training and validation data as text files
    with open("train.txt", "w") as f:
        f.write(train_text)
    with open("val.txt", "w") as f:
        f.write(val_text)

    # Set up the tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")  #also try gpt2, gpt2-large and gpt2-medium, also gpt2-xl
    model = GPT2LMHeadModel.from_pretrained("gpt2")  #also try gpt2, gpt2-large and gpt2-medium, also gpt2-xl

    # Prepare the dataset
    train_dataset = TextDataset(tokenizer=tokenizer, file_path="train.txt", block_size=128)
    val_dataset = TextDataset(tokenizer=tokenizer, file_path="val.txt", block_size=128)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # Set up the training arguments
    training_args = TrainingArguments(
        output_dir=model_output_path,
        overwrite_output_dir=True,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=100,
        save_steps=10_000,
        save_total_limit=2,
        logging_dir='./logs',
    )

    # Train the model
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )

    trainer.train()
    trainer.save_model(model_output_path)

    # Save the tokenizer
    tokenizer.save_pretrained(model_output_path)


generating a response function, inputs- trained model, tokenizer, and a prompt string as an input and generates a response string.

In [6]:
def generate_response(model, tokenizer, prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Create the attention mask and pad token id
    attention_mask = torch.ones_like(input_ids)
    pad_token_id = tokenizer.eos_token_id

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Main Function

In [8]:
def main():
  directory = "/content/drive/MyDrive/CHATBOT"
  model_output_path = "/content/drive/MyDrive/CHATBOT/gpt2"

  #training the chatbot
  train_chatbot(directory, model_output_path)

  #loading the fine-tuned model and tokenizer
  model = GPT2LMHeadModel.from_pretrained(model_output_path)
  tokenizer = GPT2Tokenizer.from_pretrained(model_output_path)

  #test the chatbot
  prompt = "What was the atomic bomb?"
  response = generate_response(model,tokenizer, prompt)
  print("Generated response: ", response)

In [9]:
if __name__=="__main__":
  main()

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,2.963000
1000,2.077200
1500,1.419700
2000,0.925100
2500,0.588600
3000,0.400000
3500,0.280500
4000,0.206900
4500,0.160300
5000,0.124800


Generated response:  What was the atomic bomb? A simple one-liners question
ard was asked of Arthur Compton in 19
41 by Arthur Compton, who got the idea from a magazine article by Pearl Buck of the Uni
versity of Illinois Press. In the front row are Norris Bradbury, John Manley, Enrico
 Fermi and J.M.B. Kellogg. Behind Manley is Oppenheimer (wearing jacket and tie), and to his left is Rich
ard


In [10]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [12]:
def genreate_response(model, tokenizer, prompt, max_length = 250):
  input_ids = tokenizer.ecode(prompt, return_tensors="pt")

  #creating attention mask and pad token id
  attention_mask = torch.ones_like(input_ids)
  pad_token_id = tokenizer.eos_token_id

  output = model.generate(
      input_ids,
      max_length=max_length,
      num_return_sequences=1,
      attention_mask=attention_mask,
      pad_token_id=pad_token_id
  )
  return tokenizer.decode(output[0], skip_special_tokens=True)

In [13]:
model_path = "/content/drive/MyDrive/CHATBOT/gpt2"
#load the fine-tuned model and tokenizer
my_chat_model = GPT2LMHeadModel.from_pretrained(model_path)
my_chat_tokenizer = GPT2Tokenizer.from_pretrained(model_path)

In [21]:
prompt = "Who was the founder of Manhattan Project"
response = generate_response(my_chat_model, my_chat_tokenizer, prompt, max_length=50)
print("Generated response:", response)

Generated response: Who was the founder of Manhattan Project?  
The key questions surrounding the Manhattan Project were: Was Oppenheimer a Soviet spy, a
nd was he a member of the British Mission?; Was he a spy in the strict sens

